# 日本語テキストデータの分類

日本語ニュースフィードのカテゴリを教師データとし、カテゴリを予測

1. テキストの段階で教師データとテストデータに分割
2. 教師データからモデルを作成
3. テストデータについて精度を検証

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

# 日本語モデル
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み、確認
feeds = pd.read_csv('data/output_jp.csv')

# title と summary を結合
# str.cat() により複数列の文字列を結合
# - sep=' ': 間に挟む文字列
# - na_rep='': NaN は空文字列に変換（指定しないと結合結果が NaN になる）
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

### 教師データの作成

URLに基づいた教師ラベルの設定：

label 0
- business
- economy
- politics
- cat4 (nhk)
- cat5 (nhk)

label 1
- culture
- science
- sport
- cat2 (nhk)
- cat7 (nhk)

label 2
- 上記以外

In [ ]:
label_0 = ['business', 'economy', 'politics', 'cat4', 'cat5']
label_1 = ['culture', 'science', 'sport', 'cat2', 'cat7']

# label_0 の単語が url の中に含まれている：0
# label_1 の単語が url の中に含まれている：1
# それ以外：2
def get_label(url):
    url = url.lower()
    # map: label_0 の単語それぞれについて、url の中に含まれているか真偽を返す
    # any: mapの結果について論理和をとる
    if any(map(lambda x: x in url, label_0)):
        return 0
    if any(map(lambda x: x in url, label_1)):
        return 1
    else:
        return 2

# df['url'] について get_label を適用した結果を df['label'] として追加
df['label'] = df['url'].map(lambda x: get_label(x))
# 各ラベルの数を確認
df['label'].value_counts()

In [ ]:
# label 0, 1 を取り出す
df = df.query('label != 2')
# 数を確認
df['label'].value_counts()

### 日本語テキストに対する前処理

- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [ ]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

# テキストの段階で教師データとテストデータに分割

In [ ]:
from sklearn.model_selection import train_test_split

text_train, text_test, Y_train, Y_test = train_test_split(df.text, df.label, test_size=0.3)

### テキストのベクトル化

- Bag of Words (BoW)

In [ ]:
# 初期化
vectorizer = CountVectorizer(tokenizer=preprocess)

# 教師データのベクトル化
vector = vectorizer.fit_transform(text_train)

# テストデータのベクトル化
vector_test = vectorizer.transform(text_test)

### 分類モデルの作成（学習）

- ナイーブベイズ分類器

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# 説明変数、目的変数
# - vector が sparse のため toarray() により dense に変換
X = vector.toarray()

# ナイーブベイズ分類器
# - 特徴量（説明変数）は整数のカウントデータ（単語の出現頻度など）
model = MultinomialNB()

# 学習
model.fit(X, Y_train)

# テストデータについて精度を検証

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve, auc

# テストデータの説明変数
X_test = vector_test.toarray()

# テストデータについてモデルからの予測値を計算
Y_predict = model.predict(X_test)

# 精度
print('正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}'.format(
    accuracy_score(Y_test, Y_predict), precision_score(Y_test, Y_predict), recall_score(Y_test, Y_predict)))
# ROC, AUC
Y_proba = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(Y_test, Y_proba[:, 1])
plt.plot(fpr, tpr, label='AUC={:.3f}'.format(auc(fpr, tpr)))
plt.xlabel('偽陽性率 (FP率)')
plt.ylabel('真陽性率 (TP率)')
plt.title('ROC曲線')
plt.legend()
plt.show()